<a href="https://colab.research.google.com/github/ABDULLAH-AHMAD-OFFICIAL/edge-connect-face-reconstruction/blob/main/colab/EdgeConnect_Face_Reconstruction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🎭 EdgeConnect Face Reconstruction

**Author:** ABDULLAH AHMAD  
**Repository:** [edge-connect-face-reconstruction](https://github.com/ABDULLAH-AHMAD-OFFICIAL/edge-connect-face-reconstruction)

Welcome to the EdgeConnect Face Reconstruction project! This notebook allows you to run advanced face inpainting directly in Google Colab with zero local setup required.

## 🌟 What this does:
- **Detects white mask regions** in facial images automatically
- **Reconstructs missing areas** using edge-guided inpainting
- **Provides high-quality results** using multiple inpainting algorithms
- **Works with any image** containing white masks (like nose masks)

## 🚀 Let's get started!

## 📦 Step 1: Setup and Installation

First, let's install all required dependencies and clone the repository.

In [ ]:
# Install required packages
!pip install torch torchvision opencv-python pillow matplotlib scikit-image numpy scipy pyyaml

# Clone the repository
!git clone https://github.com/ABDULLAH-AHMAD-OFFICIAL/edge-connect-face-reconstruction.git
%cd edge-connect-face-reconstruction

# Install the package
!pip install -e .

print("✅ Setup complete! Ready to reconstruct faces.")

## 📚 Step 2: Import Libraries

In [ ]:
import os
import numpy as np
import cv2
from PIL import Image
import matplotlib.pyplot as plt
from google.colab import files
import zipfile
import io

# Import our custom modules
from src.face_reconstructor import FaceReconstructor
from src.utils import load_image, save_image

# Configure matplotlib for better display
plt.rcParams['figure.figsize'] = (15, 10)
plt.rcParams['image.interpolation'] = 'bilinear'

print("📚 Libraries imported successfully!")

## 🎨 Step 3: Quick Demo with Sample Image

Let's start with a quick demo using a sample image to see how the reconstruction works.

In [ ]:
# Initialize the face reconstructor
reconstructor = FaceReconstructor()

# Create a sample image with white nose mask
print("🎭 Creating sample face with white nose mask...")
sample_path = reconstructor.create_sample_image(size=256, output_path='sample_face.jpg')

# Display the sample image
sample_img = load_image(sample_path)
plt.figure(figsize=(8, 8))
plt.imshow(sample_img)
plt.title('Sample Face with White Nose Mask', fontsize=16)
plt.axis('off')
plt.show()

print("✅ Sample image created!")

In [ ]:
# Process the sample image
print("🔄 Processing sample image...")
results = reconstructor.process_image(sample_path, output_dir='./demo_output')

# Visualize the results
reconstructor.visualize_results(results, figsize=(18, 12))

print("✅ Demo completed! See the reconstruction results above.")

## 📤 Step 4: Upload Your Own Image

Now let's process your own image! Upload an image with white mask regions that you want to reconstruct.

In [ ]:
# Upload your image
print("📤 Please upload your image with white mask regions:")
uploaded = files.upload()

# Get the uploaded file
uploaded_filename = list(uploaded.keys())[0]
print(f"✅ Uploaded: {uploaded_filename}")

# Display the uploaded image
uploaded_img = load_image(uploaded_filename)
plt.figure(figsize=(10, 8))
plt.imshow(uploaded_img)
plt.title(f'Your Uploaded Image: {uploaded_filename}', fontsize=16)
plt.axis('off')
plt.show()

print(f"Image size: {uploaded_img.shape[1]}x{uploaded_img.shape[0]} pixels")

## ⚙️ Step 5: Configure Parameters (Optional)

Adjust these parameters if needed for better results with your specific image.

In [ ]:
# Configuration parameters - adjust as needed
config = {
    'threshold': 240,        # White mask detection threshold (0-255)
    'edge_sigma': 2,         # Edge detection blur strength
    'edge_weight': 0.3,      # Edge guidance weight (0-1)
    'inpaint_radius': 3,     # Inpainting radius
    'target_size': (512, 512),  # Target processing size (optional)
    'save_intermediate': True,
    'output_quality': 95
}

# Update reconstructor with new config
reconstructor.update_config(**config)

print("⚙️ Configuration updated:")
for key, value in config.items():
    print(f"  {key}: {value}")

print("\n💡 Tips for parameter adjustment:")
print("  • Lower threshold (200-220) for lighter masks")
print("  • Higher edge_weight (0.4-0.6) for more edge emphasis")
print("  • Larger target_size for higher quality results")

## 🎯 Step 6: Process Your Image

Let's reconstruct your image using the configured parameters!

In [ ]:
try:
    print(f"🔄 Processing your image: {uploaded_filename}")
    print("This may take a few moments...")
    
    # Process the uploaded image
    results = reconstructor.process_image(
        uploaded_filename, 
        output_dir='./your_results',
        target_size=config.get('target_size')
    )
    
    print("✅ Processing completed!")
    
    # Display results
    print("\n📊 Reconstruction Results:")
    reconstructor.visualize_results(results, figsize=(20, 14))
    
    # Print statistics
    mask_pixels = np.sum(results['mask'] > 0)
    total_pixels = results['mask'].size
    mask_percentage = (mask_pixels / total_pixels) * 100
    
    print(f"\n📈 Statistics:")
    print(f"  • Mask pixels detected: {mask_pixels:,}")
    print(f"  • Mask coverage: {mask_percentage:.2f}% of image")
    print(f"  • Image dimensions: {results['original'].shape[1]}x{results['original'].shape[0]}")
    
except ValueError as e:
    print(f"❌ Error: {e}")
    print("\n💡 Try adjusting the threshold parameter:")
    print("  • Lower threshold (200-220) for lighter masks")
    print("  • Check if your image actually contains white regions")
except Exception as e:
    print(f"❌ Unexpected error: {e}")
    print("Please check your image format and try again.")

## 💾 Step 7: Download Results

Download all the generated results including the reconstructed image, mask, and comparison images.

In [ ]:
# Create a zip file with all results
def create_results_zip():
    zip_buffer = io.BytesIO()
    
    with zipfile.ZipFile(zip_buffer, 'w', zipfile.ZIP_DEFLATED) as zip_file:
        # Add demo results
        demo_dir = './demo_output'
        if os.path.exists(demo_dir):
            for file in os.listdir(demo_dir):
                file_path = os.path.join(demo_dir, file)
                if os.path.isfile(file_path):
                    zip_file.write(file_path, f'demo/{file}')
        
        # Add user results
        user_dir = './your_results'
        if os.path.exists(user_dir):
            for file in os.listdir(user_dir):
                file_path = os.path.join(user_dir, file)
                if os.path.isfile(file_path):
                    zip_file.write(file_path, f'your_results/{file}')
    
    zip_buffer.seek(0)
    return zip_buffer.getvalue()

# Create and download the zip file
print("📦 Creating download package...")
zip_data = create_results_zip()

# Download the results
files.download(io.BytesIO(zip_data), 'face_reconstruction_results.zip')
print("✅ Results downloaded as 'face_reconstruction_results.zip'")

print("\n📁 Your download contains:")
print("  • demo/ - Sample reconstruction results")
print("  • your_results/ - Your image reconstruction results")
print("    ├── *_reconstructed.jpg - Final result")
print("    ├── *_mask.jpg - Detected mask")
print("    ├── *_edges.jpg - Edge detection")
    ├── *_comparison.jpg - Before vs after")

## 🔄 Step 8: Batch Processing (Optional)

If you have multiple images to process, upload them all and process them in batch.

In [ ]:
# Upload multiple images for batch processing
print("📤 Upload multiple images for batch processing (optional):")
print("You can select multiple files at once!")

batch_uploaded = files.upload()

if batch_uploaded:
    print(f"✅ Uploaded {len(batch_uploaded)} images for batch processing")
    
    # Create input directory
    os.makedirs('./batch_input', exist_ok=True)
    
    # Move uploaded files to input directory
    for filename in batch_uploaded.keys():
        os.rename(filename, f'./batch_input/{filename}')
    
    # Process batch
    print("🔄 Processing batch...")
    reconstructor.batch_process('./batch_input', './batch_output')
    
    print("✅ Batch processing completed!")
    
    # Show summary
    output_files = os.listdir('./batch_output')
    print(f"📊 Generated {len(output_files)} output files")
else:
    print("ℹ️ No files uploaded for batch processing")

## 🔧 Step 9: Advanced Configuration & Tips

Here are some advanced tips and parameter explanations for getting the best results.

In [ ]:
print("🔧 Advanced Configuration Tips:")
print("\n🎯 Threshold Parameter (threshold):")
print("  • 240-255: Very selective, only pure white")
print("  • 220-240: Good for most white masks")
print("  • 200-220: More inclusive, catches light gray")
print("  • 180-200: Very inclusive, may catch shadows")

print("\n🔍 Edge Detection (edge_sigma):")
print("  • 1-2: Sharp edges, good for clear images")
print("  • 2-3: Balanced, works for most images")
print("  • 3-5: Smoother edges, good for noisy images")

print("\n⚖️ Edge Weight (edge_weight):")
print("  • 0.1-0.3: Subtle edge guidance")
print("  • 0.3-0.5: Balanced approach (recommended)")
print("  • 0.5-0.8: Strong edge emphasis")

print("\n📏 Image Size (target_size):")
print("  • (256, 256): Fast processing, lower quality")
print("  • (512, 512): Balanced speed/quality")
print("  • (1024, 1024): High quality, slower processing")
print("  • None: Use original size")

print("\n💡 Troubleshooting:")
print("  • 'No white regions detected': Lower threshold")
print("  • Poor edge detection: Adjust edge_sigma")
print("  • Blurry results: Increase target_size")
print("  • Artifacts in result: Try different edge_weight")

# Example of parameter testing
print("\n🧪 Want to test different parameters?")
print("Modify the config dictionary in Step 5 and re-run Step 6!")

## ℹ️ About This Project

### 🎯 What is EdgeConnect Face Reconstruction?

This project implements an EdgeConnect-inspired approach for reconstructing masked facial regions. It's particularly effective for:

- **Medical masks and face coverings**
- **Privacy protection applications**
- **Photo restoration**
- **Digital art and creative projects**

### 🔬 How it works:

1. **Mask Detection**: Automatically identifies white/light regions
2. **Edge Analysis**: Uses Canny edge detection to understand structure
3. **Edge Propagation**: Extends edges into masked regions
4. **Inpainting**: Combines multiple algorithms for natural results
5. **Blending**: Merges results using edge guidance

### 👨‍💻 Author

**ABDULLAH AHMAD**  
GitHub: [@ABDULLAH-AHMAD-OFFICIAL](https://github.com/ABDULLAH-AHMAD-OFFICIAL)  
Repository: [edge-connect-face-reconstruction](https://github.com/ABDULLAH-AHMAD-OFFICIAL/edge-connect-face-reconstruction)

### 📄 License

This project is licensed under the MIT License - see the [LICENSE](https://github.com/ABDULLAH-AHMAD-OFFICIAL/edge-connect-face-reconstruction/blob/main/LICENSE) file for details.

### 🙏 Acknowledgments

- Original EdgeConnect paper: [Nazeri et al., 2019](https://arxiv.org/abs/1901.00212)
- OpenCV community for inpainting algorithms

---

⭐ **If you found this helpful, please star the repository!** ⭐

🐛 **Found a bug or have suggestions?** [Open an issue](https://github.com/ABDULLAH-AHMAD-OFFICIAL/edge-connect-face-reconstruction/issues)